   # TP53: Effects of Mutation on Interacting Proteins

<b>Standard imports for playing with and plotting data frames.</b>

In [1]:
import pandas as pd
import numpy as np
import re
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

<b>Import CPTAC data</b>

In [2]:
import CPTAC

Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [3]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

### List of proteins that interact with TP53 (according to Uniprot and String)

In [4]:
gene = 'TP53'
protList = ['TGFB1', 'AXIN1', 'EP300', 'HRMT1L2', 'CARM1', 'TAF1', 'ING4', 'CABLES1', 
            'TP73', 'HIPK1', 'HIPK2', 'TP53INP1', 'TP53BP', 'WWOX', 'HCV', 'USP7', 'SYVN1', 'HSP90AB1',
           'CHD8', 'ARMC10', 'BANP', 'CDKN2AIP', 'NUAK1', 'STK11', 'LKB1', 'UHRF2', 'E4F1', 'YWHAZ',
           'MAML1', 'MKRN1', 'PML', 'MDM2', 'FBXO42', 'ATM', 'PP2R2A', 'AURKA', 'DAXX', 'BRD7', 'TRIM24',
           'L3MBTL1', 'GRK5', 'CAK', 'PTK2B', 'PYK2', 'MDM2', 'PRKCG', 'PPIF', 'KAT6A', 'UBC9', 'ZNF385B',
           'ZNF385A', 'ANKRD2', 'RFFL', 'RNF34', 'MTA1', 'COP1', 'CCAR2', 'MORC3', 'POU4F2', 'AFG1L', 'UBD',
           'TAF6', 'HPV', 'UL123', 'E1B-55K', 'BAX', 'FAS', 'BCL2', 'CREBBP', 'CDK2', 'MDM2', 'CDKN2A', 'ATM',
            'CCNB1', 'TERT', 'IGF1R', 'ALB', 'MMP1']

### t-test for proteome abundance of these proteins when TP53 is mutated
<b>Note: We are only interested in looking at proteins whose abundance is significantly affected by TP53 mutation, and we must do a Bonferroni correction on our p-value</b>

In [5]:
pcutoff = 0.05 / len(protList)
#pcutoff = 1

In [12]:
significantResults = [];
for protein in protList: 
    if protein in proteomics.columns:
        cross = CPTAC.compare_mutations(proteomics, protein, gene)
        cross = cross.loc[cross['Patient_Type'] == 'Tumor'][["Mutation", protein]].dropna(axis=0)
        mutated = cross.loc[cross["Mutation"] != "Wildtype"]
        wt = cross.loc[cross["Mutation"] == "Wildtype"]
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        if(ttest[1] <= pcutoff):
            significantResults.append(protein)
            print("Test for " + protein + ": ")
            print(ttest) 

Test for CABLES1: 
Ttest_indResult(statistic=-4.533885225055603, pvalue=1.6398425638878636e-05)
Test for HSP90AB1: 
Ttest_indResult(statistic=3.775761395509814, pvalue=0.000273573248009503)
Test for CHD8: 
Ttest_indResult(statistic=4.65599574118626, pvalue=1.0132380205138297e-05)
Test for AURKA: 
Ttest_indResult(statistic=4.2121494006965365, pvalue=8.335995102753076e-05)
Test for DAXX: 
Ttest_indResult(statistic=4.139121723641875, pvalue=7.383403320014249e-05)
Test for FAS: 
Ttest_indResult(statistic=-5.5103825350041875, pvalue=2.894104669939677e-07)
Test for CDKN2A: 
Ttest_indResult(statistic=4.344971931273746, pvalue=3.583610475413602e-05)
Test for CCNB1: 
Ttest_indResult(statistic=4.205828399957831, pvalue=5.7589441814949935e-05)


### List of significantly affected proteins

In [45]:
print(significantResults)

['TGFB1', 'AXIN1', 'EP300', 'CARM1', 'TAF1', 'ING4', 'CABLES1', 'WWOX', 'USP7', 'SYVN1', 'HSP90AB1', 'CHD8', 'ARMC10', 'BANP', 'CDKN2AIP', 'STK11', 'UHRF2', 'E4F1', 'YWHAZ', 'MAML1', 'MKRN1', 'PML', 'FBXO42', 'ATM', 'AURKA', 'DAXX', 'BRD7', 'TRIM24', 'GRK5', 'PTK2B', 'PPIF', 'KAT6A', 'ZNF385A', 'RFFL', 'RNF34', 'MTA1', 'COP1', 'CCAR2', 'MORC3', 'AFG1L', 'TAF6', 'BAX', 'FAS', 'BCL2', 'CREBBP', 'CDK2', 'CDKN2A', 'ATM', 'CCNB1', 'IGF1R', 'ALB']


### t-test for effects of frame shift and nonsense mutations of TP53 on protein levels of the above 5 proteins

In [46]:
significantResults.append(gene)
for protein in significantResults: 
    if protein in proteomics.columns:
        cross = CPTAC.merge_mutations(proteomics, protein, gene)
        cross = cross[:100]
        cross = cross[["Mutation", protein]].dropna(axis=0)
        mutated = cross.loc[(cross["Mutation"] == "Frame_Shift_Del") | 
                            (cross["Mutation"] == "Frame_Shift_Ins") |
                            (cross["Mutation"] == 'Nonsense_Mutation')]
        wt = cross.loc[cross["Mutation"] == "Wildtype"]
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        print("Test for " + protein + ": ")
        print(ttest) 

Test for TGFB1: 
Ttest_indResult(statistic=1.9825129599758367, pvalue=0.05081216790049328)
Test for AXIN1: 
Ttest_indResult(statistic=-1.427023770784033, pvalue=0.15884410766512902)
Test for EP300: 
Ttest_indResult(statistic=1.2692974888346118, pvalue=0.20796997367491624)
Test for CARM1: 
Ttest_indResult(statistic=0.24930799253313413, pvalue=0.8037536664864771)
Test for TAF1: 
Ttest_indResult(statistic=2.7048130253722187, pvalue=0.008328380771354348)
Test for ING4: 
Ttest_indResult(statistic=-0.07148071091431717, pvalue=0.9432313416673633)
Test for CABLES1: 
Ttest_indResult(statistic=-3.7356442692900407, pvalue=0.0003471526801904925)
Test for WWOX: 
Ttest_indResult(statistic=0.6793408809482863, pvalue=0.4988586179525666)
Test for USP7: 
Ttest_indResult(statistic=0.6953698472987885, pvalue=0.48881372939754775)
Test for SYVN1: 
Ttest_indResult(statistic=0.5628496546973896, pvalue=0.5750918895987001)
Test for HSP90AB1: 
Ttest_indResult(statistic=2.566923200743245, pvalue=0.012100611516942

### Build the dataframe for plotting

In [47]:
plotdf = CPTAC.merge_mutations(proteomics, gene)[['Mutation', gene]]

for protein in significantResults:
    if(protein is not 'TP53'):
        proteindf = CPTAC.merge_mutations(proteomics, protein, gene)[[protein]]
        plotdf = plotdf.join(proteindf)

ValueError: columns overlap but no suffix specified: Index(['ATM'], dtype='object')

In [ ]:
plotdf = plotdf.replace({'Mutation': {'Frame_Shift_Del': 'Truncating',
                                      'Frame_Shift_Ins': 'Truncating',
                                      'Nonsense_Mutation': 'Truncating',
                                      'Wildtype': 'Control',
                                      'Missense_Mutation': 'Missense'
                                     }})

In [ ]:
plotdf = pd.melt(plotdf, id_vars=['Mutation'], var_name='Protein', value_name='Proteomics')
plotdf = plotdf.loc[(plotdf["Mutation"] == "Control") | (plotdf["Mutation"] == "Missense") | (plotdf["Mutation"] == "Truncating")]
plotdf = plotdf.dropna(axis=0)

In [ ]:
a4_dims = (23.4, 16.54)
fig, ax = plt.subplots(figsize=a4_dims)
my_pal = {"Missense": "#6C906D", "Control": "#596B94", "Truncating": "#A35857"}

#Create the plot
boxplt = sns.boxplot(x='Protein', y='Proteomics', hue='Mutation', data=plotdf, palette=my_pal)
boxplt = sns.stripplot(data=plotdf, x='Protein', y='Proteomics', hue='Mutation', dodge=True, jitter=True, color='.3')

#Add styling
boxplt.set_xlabel('')
boxplt.set_ylabel('Protein Level', fontsize='40')
boxplt.tick_params(labelsize='20')

#Adjust legend
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles[0:3], ['Missense', 'Control', 'Truncating'], title='TP53 Status', fontsize='20', frameon=False)
boxplt.get_legend().get_title().set_fontsize('20')
boxplt.get_legend().set_bbox_to_anchor((0.15, 0.15, 0, 0))

#Add significance indicator: TP53/Missense
x1, x2 = -0.25, -0.05
y, h, col = 4.45, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: TP53/Truncation
x1, x2 = 0.05, 0.25
y, h, col = 4.25, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "ns", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: CABLES1/Missense
x1, x2 = 0.75, 0.95
y, h, col = 1.3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "***", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: CABLES1/Truncation
x1, x2 = 1.05, 1.25
y, h, col = 0.8 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: HSP90AB1/Missense
x1, x2 = 1.75, 1.95
y, h, col = 1 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: HSP90AB1/Truncation
x1, x2 = 2.05, 2.25
y, h, col = 0.8 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "***", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: CHD8/Missense
x1, x2 = 2.75, 2.95
y, h, col = 1 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: CHD8/Truncation
x1, x2 = 3.05, 3.25
y, h, col = 0.8 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: UHRF2/Missense
x1, x2 = 3.75, 3.95
y, h, col = 1 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: UHRF2/Truncation
x1, x2 = 4.05, 4.25
y, h, col = 0.8 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "ns", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: AURKA/Missense
x1, x2 = 4.75, 4.95
y, h, col = 3 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "****", ha='center', va='bottom', color=col, fontsize='23')

#Add significance indicator: AURKA/Truncation
x1, x2 = 5.05, 5.25
y, h, col = 2.8 + .3, .3, 'k'
plt.plot([x1, x1, x2, x2], [y+h, y+h, y+h, y+h], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "*", ha='center', va='bottom', color=col, fontsize='23')

fig = boxplt.get_figure()
fig.savefig('p53_trans.png', dpi=300)
plt.show()

## Phosphoproteome abundance of interacting proteins

### This is the same list as before, with some of the end numbers of the proteins removed; less specificity in protein names captures more potentially significant results in the phosphoproteomic data

In [ ]:
phosProtList = ['TGFB', 'AXIN1', 'EP300', 'HRMT1L2', 'CARM1', 'TAF1', 'ING4', 'CABLES1', 
            'TP73', 'HIPK1', 'HIPK2', 'TP53INP1', 'TP53BP', 'WWOX', 'HCV', 'USP7', 'SYVN1', 'HSP90AB1',
           'CHD8', 'ARMC10', 'BANP', 'CDKN2AIP', 'NUAK1', 'STK11', 'LKB1', 'UHRF2', 'E4F1', 'YWHAZ',
           'MAML1', 'MKRN1', 'PML', 'MDM2', 'FBXO42', 'ATM', 'PP2R2A', 'AURKA', 'DAXX', 'BRD7', 'TRIM24',
           'L3MBTL1', 'GRK5', 'CAK', 'PTK2B', 'PYK2', 'MDM2', 'PRKCG', 'PPIF', 'KAT6A', 'UBC9', 'ZNF385B',
           'ZNF385A', 'ANKRD2', 'RFFL', 'RNF34', 'MTA1', 'COP1', 'CCAR2', 'MORC3', 'POU4F2', 'AFG1L', 'UBD',
           'TAF6', 'HPV', 'UL123', 'E1B-55K', 'BAX', 'FAS', 'BCL2', 'CREBBP', 'CDK2', 'MDM2', 'CDKN2A', 
            'CDKN1A', 'ATM', 'MDM4']

In [ ]:
sites = phos.columns;
significantPhosResults = [];
totalSites = 0

for protein in phosProtList:
    matchesdf = phos.filter(regex=protein)
    totalSites += len(matchesdf.columns)

pcutoff = 0.05 / totalSites   

for protein in phosProtList:
    pattern = re.compile(protein)
    isInList = filter(pattern.search, sites)
    if next(isInList, None) is not None:
        phosphositesdf = CPTAC.merge_mutations(phos, protein, gene)
        for site in phosphositesdf.columns:
            if (site is not 'Mutation'):
                sitedf = CPTAC.merge_mutations(phos, site, gene)
                mutateddf = sitedf.loc[sitedf['Mutation'] == 'Missense_Mutation'].dropna(axis=0)
                wtdf = sitedf.loc[sitedf['Mutation'] == 'Wildtype'].dropna(axis=0)
                ttest = scipy.stats.ttest_ind(mutateddf[site], wtdf[site])
                if(ttest[1] <= pcutoff):
                    significantPhosResults.append(site)
                    print("Test for " + site + ": ")
                    print(ttest) 

### List of significantly affected phosphorylation sites

In [ ]:
print(significantPhosResults)